<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Regular_Season_Batter" data-toc-modified-id="Regular_Season_Batter-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Regular_Season_Batter</a></span></li><li><span><a href="#preprocessing" data-toc-modified-id="preprocessing-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>preprocessing</a></span><ul class="toc-item"><li><span><a href="#weight,-height" data-toc-modified-id="weight,-height-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>weight, height</a></span></li><li><span><a href="#age" data-toc-modified-id="age-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>age</a></span></li><li><span><a href="#starting-salary" data-toc-modified-id="starting-salary-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>starting salary</a></span></li><li><span><a href="#position,-team" data-toc-modified-id="position,-team-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>position, team</a></span></li><li><span><a href="#drop-features" data-toc-modified-id="drop-features-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>drop features</a></span></li><li><span><a href="#missing-value" data-toc-modified-id="missing-value-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>missing value</a></span></li><li><span><a href="#누적-시즌-참여-횟수" data-toc-modified-id="누적-시즌-참여-횟수-2.7"><span class="toc-item-num">2.7&nbsp;&nbsp;</span>누적 시즌 참여 횟수</a></span></li><li><span><a href="#시즌별-최대-연속-출장-경기" data-toc-modified-id="시즌별-최대-연속-출장-경기-2.8"><span class="toc-item-num">2.8&nbsp;&nbsp;</span>시즌별 최대 연속 출장 경기</a></span></li><li><span><a href="#역대-OPS-추세" data-toc-modified-id="역대-OPS-추세-2.9"><span class="toc-item-num">2.9&nbsp;&nbsp;</span>역대 OPS 추세</a></span></li><li><span><a href="#역대-OBP-추세" data-toc-modified-id="역대-OBP-추세-2.10"><span class="toc-item-num">2.10&nbsp;&nbsp;</span>역대 OBP 추세</a></span></li><li><span><a href="#역대-SLG-추세" data-toc-modified-id="역대-SLG-추세-2.11"><span class="toc-item-num">2.11&nbsp;&nbsp;</span>역대 SLG 추세</a></span></li><li><span><a href="#역대-OPS-상승-수-하락-수" data-toc-modified-id="역대-OPS-상승-수-하락-수-2.12"><span class="toc-item-num">2.12&nbsp;&nbsp;</span>역대 OPS 상승 수 하락 수</a></span></li><li><span><a href="#역대-OBP-상승-수-하락-수" data-toc-modified-id="역대-OBP-상승-수-하락-수-2.13"><span class="toc-item-num">2.13&nbsp;&nbsp;</span>역대 OBP 상승 수 하락 수</a></span></li><li><span><a href="#역대-SLG-상승-수-하락-수" data-toc-modified-id="역대-SLG-상승-수-하락-수-2.14"><span class="toc-item-num">2.14&nbsp;&nbsp;</span>역대 SLG 상승 수 하락 수</a></span></li></ul></li><li><span><a href="#test-set" data-toc-modified-id="test-set-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>test set</a></span></li></ul></div>

In [132]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder

import xgboost as xgb

In [133]:
def dummy_feature(df, variables):
    dummies = pd.DataFrame()
    for var in variables:
        dummy = pd.get_dummies(df[var], prefix=var, drop_first=True)
        dummies = pd.concat([dummies, dummy], axis=1)
        df = df.drop(var, axis=1)
    df = pd.concat([df, dummies], axis=1)
    return df

# Regular_Season_Batter

In [143]:
data1 = pd.read_csv('../dataset/Regular_Season_Batter.csv', index_col=0)
data2 = pd.read_csv('../dataset/Regular_Season_Batter_Day_by_Day.csv', index_col=0)

data_temp = data1.copy()
data1.shape

(2454, 28)

In [135]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2454 entries, 0 to 344
Data columns (total 28 columns):
batter_name        2454 non-null object
year               2454 non-null int64
team               2454 non-null object
avg                2428 non-null float64
G                  2454 non-null int64
AB                 2454 non-null int64
R                  2454 non-null int64
H                  2454 non-null int64
2B                 2454 non-null int64
3B                 2454 non-null int64
HR                 2454 non-null int64
TB                 2454 non-null int64
RBI                2454 non-null int64
SB                 2454 non-null int64
CS                 2454 non-null int64
BB                 2454 non-null int64
HBP                2454 non-null int64
SO                 2454 non-null int64
GDP                2454 non-null int64
SLG                2428 non-null float64
OBP                2430 non-null float64
E                  2454 non-null int64
height/weight      1652 non-

# preprocessing

In [136]:
def OPS_nan_(data):
    data = data.fillna(0)
    return data

## weight, height
- 키, 몸무게가 결측치인 선수가 101명...

In [144]:
def weight_height_(data):
    data['height'] = data['height/weight'].apply(lambda x: np.nan if str(x) == 'nan' else x.split('/')[0][:3]).astype(float)
    data['weight'] = data['height/weight'].apply(lambda x: np.nan if str(x) == 'nan' else x.split('/')[1][:-2]).astype(float)
    data.height = data.height.fillna(np.median(data.height.dropna()))
    data.weight = data.weight.fillna(np.median(data.weight.dropna()))
    data = data.drop('height/weight', axis=1)
    return data

In [146]:
data_temp[data_temp['height/weight'].isnull()]['batter_name'].nunique()

101

In [147]:
data_temp = weight_height_(data_temp)

## age

In [148]:
def age_(data):
    data['age'] = data.year - data.year_born.apply(lambda x: int(x[:4]))
    data = data.drop('year_born', axis=1)
    return data

In [149]:
data_temp = age_(data_temp)

## starting salary

In [150]:
def starting_salary_(data):
    data.starting_salary = data.starting_salary.apply(lambda x: x[:-2] if str(x)!='nan' else np.nan).astype(float)
    data.starting_salary = data.starting_salary.fillna(np.median(data.starting_salary.dropna()))
    return data

In [151]:
data_temp = starting_salary_(data_temp)

## position, team

In [152]:
def categorical_variables(data, features):
    data = dummy_feature(data, features)
    return data

In [153]:
cat_features = ['position','team']
data_temp = categorical_variables(data_temp, cat_features)

## drop features

In [154]:
def drop_variables(data, features):
    data = data.drop(features, axis=1)
    return data

In [155]:
drop_features = ['career']
data_temp = drop_variables(data_temp, drop_features)

## missing value

**나머지 결측값은 모두 0으로 처리한다.**
- 이유는 타수가 0 이여서 OPS,SLG(장타율)는 0이고 OBP(출루율)는 타수가 아닌 사유로 생기는 경우가 있기 때문에 NaN이 아닌 값도 있다.
- 출루율이 1인경우는 볼넷

In [157]:
features = ['avg','G','AB','R','H','2B','3B','HR','TB','RBI','SB','CS','BB','HBP','SO','GDP','SLG','OBP','E','OPS']
data_temp[data_temp.SLG.isnull() | data_temp.OBP.isnull()].loc[:,features]

,avg,G,AB,R,H,2B,3B,HR,TB,RBI,SB,CS,BB,HBP,SO,GDP,SLG,OBP,E,OPS
batter_id,,,,,,,,,,,,,,,,,,,,
138,NaN,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN
21,NaN,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN
24,NaN,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,NaN,NaN,0,NaN
31,NaN,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN
45,NaN,3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,NaN,NaN,0,NaN
97,NaN,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,NaN,NaN,0,NaN
132,NaN,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN
151,NaN,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN
173,NaN,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN


In [158]:
data_temp = OPS_nan_(data_temp)

## 누적 시즌 참여 횟수

In [160]:
def cum_season_(data, names):
    for name in names:
        data.loc[data.batter_name==name,'cum_season'] = np.arange(data[data.batter_name==name].shape[0]) 
    return data

In [161]:
names = data_temp.batter_name.unique()
data_temp = cum_season_(data_temp, names)

## 시즌별 최대 연속 출장 경기

In [87]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109771 entries, 0 to 332
Data columns (total 19 columns):
batter_name      109771 non-null object
date             109771 non-null float64
opposing_team    109771 non-null object
avg1             109771 non-null object
AB               109771 non-null int64
R                109771 non-null int64
H                109771 non-null int64
2B               109771 non-null int64
3B               109771 non-null int64
HR               109771 non-null int64
RBI              109771 non-null int64
SB               109771 non-null int64
CS               109771 non-null int64
BB               109771 non-null int64
HBP              109771 non-null int64
SO               109771 non-null int64
GDP              109771 non-null int64
avg2             109771 non-null float64
year             109771 non-null int64
dtypes: float64(2), int64(14), object(3)
memory usage: 16.7+ MB


In [91]:
data2 = pd.merge(data1[['batter_name','year','team']], data2, on=['batter_name','year'], how='right')

## 역대 OPS 추세
$$OPS추세=\sum_{i=1}^n \frac{Year_{n-(i-1)}}{Year_{n}}\frac{OPS_{n-(i-1)}}{OPS_{n-i}}$$ 
- 시즌 참가 이력이 1회면 OPS_trend = 0
- 이전 시즌 OPS가 0이면 OPS_trend = 0
- OPS rate가 1미만이면(하락했으면) -OPS rate

In [260]:
def OPS_trend_(data, names):
    for name in names:
        data_batter = data[data.batter_name==name][['year','OPS']]
        OPS_trend = list()
        if data_batter.shape[0] == 1:
            OPS_trend.append(0)
        else:
            for y in range(data_batter.shape[0]):
                data_ops = data_batter.iloc[:y+1,:]
                OPS_weight = 0
                if data_ops.shape[0]!=1:
                    for i in range(data_ops.shape[0]):
                        if i == 0:
                            continue
                        if data_ops.OPS.iloc[i] == 0:
                            continue
                        elif data_ops.OPS.iloc[i-1] == 0:
                            continue
                        else:
                            ops_rate = (data_ops.OPS.iloc[i]/data_ops.OPS.iloc[i-1])
                            if ops_rate < 1:
                                ops_rate = -ops_rate
                            OPS_weight += (data_ops.year.iloc[i]/max(data_ops.year)) * ops_rate
                OPS_trend.append(np.round(OPS_weight,3))
        data.loc[data.batter_name==name,'OPS_trend'] = OPS_trend
    return data

In [170]:
data_temp = OPS_trend_(data_temp, names)

In [171]:
data_temp[data_temp.batter_name=='양의지'][['batter_name','year','OPS','OPS_trend']]

,batter_name,year,OPS,OPS_trend
batter_id,,,,
174,양의지,2007,0.000,0.000
174,양의지,2010,0.819,0.000
174,양의지,2011,0.761,-0.929
174,양의지,2012,0.748,-1.912
174,양의지,2013,0.714,-2.865
174,양의지,2014,0.840,-1.687
174,양의지,2015,0.928,-0.582
174,양의지,2016,0.973,0.467
174,양의지,2017,0.814,-0.370


In [172]:
data_temp[data_temp.batter_name=='황재균'][['batter_name','year','OPS','OPS_trend']]

,batter_name,year,OPS,OPS_trend
batter_id,,,,
342,황재균,2007,0.698,0.000
342,황재균,2008,0.567,-0.812
342,황재균,2009,0.802,0.603
342,황재균,2010,0.653,-0.212
342,황재균,2011,0.805,1.021
342,황재균,2012,0.681,0.174
342,황재균,2013,0.739,1.260
342,황재균,2014,0.863,2.427
342,황재균,2015,0.871,3.435


## 역대 OBP 추세

In [261]:
def OBP_trend_(data, names):
    for name in names:
        data_batter = data[data.batter_name==name][['year','OBP']]
        OBP_trend = list()
        if data_batter.shape[0] == 1:
            OBP_trend.append(0)
        else:
            for y in range(data_batter.shape[0]):
                data_OBP = data_batter.iloc[:y+1,:]
                OBP_weight = 0
                if data_OBP.shape[0]!=1:
                    for i in range(data_OBP.shape[0]):
                        if i == 0:
                            continue
                        if data_OBP.OBP.iloc[i] == 0:
                            continue
                        elif data_OBP.OBP.iloc[i-1] == 0:
                            continue
                        else:
                            OBP_rate = (data_OBP.OBP.iloc[i]/data_OBP.OBP.iloc[i-1])
                            if OBP_rate < 1:
                                OBP_rate = -OBP_rate
                            OBP_weight += (data_OBP.year.iloc[i]/max(data_OBP.year)) * OBP_rate
                OBP_trend.append(np.round(OBP_weight,3))
        data.loc[data.batter_name==name,'OBP_trend'] = OBP_trend
    return data

In [262]:
%%time
data_temp = OBP_trend_(data_temp, names)

Wall time: 2.59 s


## 역대 SLG 추세

In [263]:
def SLG_trend_(data, names):
    for name in names:
        data_batter = data[data.batter_name==name][['year','SLG']]
        SLG_trend = list()
        if data_batter.shape[0] == 1:
            SLG_trend.append(0)
        else:
            for y in range(data_batter.shape[0]):
                data_SLG = data_batter.iloc[:y+1,:]
                SLG_weight = 0
                if data_SLG.shape[0]!=1:
                    for i in range(data_SLG.shape[0]):
                        if i == 0:
                            continue
                        if data_SLG.SLG.iloc[i] == 0:
                            continue
                        elif data_SLG.SLG.iloc[i-1] == 0:
                            continue
                        else:
                            SLG_rate = (data_SLG.SLG.iloc[i]/data_SLG.SLG.iloc[i-1])
                            if SLG_rate < 1:
                                SLG_rate = -SLG_rate
                            SLG_weight += (data_SLG.year.iloc[i]/max(data_SLG.year)) * SLG_rate
                SLG_trend.append(np.round(SLG_weight,3))
        data.loc[data.batter_name==name,'SLG_trend'] = SLG_trend
    return data

In [264]:
%%time
data_temp = SLG_trend_(data_temp, names)

Wall time: 2.5 s


## 역대 OPS 상승 수 하락 수
- 시즌 참가 이력이 1회면 OPS_up, OPS_down은 0
- 역대 시즌 OPS가 모두 0 이면 OPS_up, OPS_down은 0
    - 이전 시즌 OPS가 0 이면 역대 OPS중 가장 최근 OPS를 기준으로 함

In [253]:
def OPS_up_down(data, names):
    for name in names:
        data_name = data[data.batter_name==name]
        for i in range(data_name.shape[0]):
            OPS_up, OPS_down = 0, 0
            year = data_name.year.iloc[i]
            data_ = data_name.iloc[:i+1,:]
            if data_.shape[0] == 1:
                data.loc[(data.batter_name==name) & (data.year==year),'OPS_up'] = OPS_up
                data.loc[(data.batter_name==name) & (data.year==year),'OPS_down'] = OPS_down
                continue
            for j in range(data_.shape[0]):
                pre_OPS = [ops for ops in data_.iloc[:j,:]['OPS'] if ops != 0]
                if j == 0:
                    continue
                elif data_.iloc[j,:]['OPS'] == 0:
                    continue
                elif len(pre_OPS) == 0:
                    continue
                up_down = data_.iloc[j,:]['OPS'] / pre_OPS[-1]
                if up_down > 1:
                    OPS_up += 1
                elif up_down < 1:
                    OPS_down += 1
                
            data.loc[(data.batter_name==name) & (data.year==year),'OPS_up'] = OPS_up
            data.loc[(data.batter_name==name) & (data.year==year),'OPS_down'] = OPS_down
    
    return data

In [254]:
%%time
data_temp = OPS_up_down(data_temp, names)

Wall time: 17.9 s


## 역대 OBP 상승 수 하락 수

In [256]:
def OBP_up_down(data, names):
    for name in names:
        data_name = data[data.batter_name==name]
        for i in range(data_name.shape[0]):
            OBP_up, OBP_down = 0, 0
            year = data_name.year.iloc[i]
            data_ = data_name.iloc[:i+1,:]
            if data_.shape[0] == 1:
                data.loc[(data.batter_name==name) & (data.year==year),'OBP_up'] = OBP_up
                data.loc[(data.batter_name==name) & (data.year==year),'OBP_down'] = OBP_down
                continue
            for j in range(data_.shape[0]):
                pre_OBP = [OBP for OBP in data_.iloc[:j,:]['OBP'] if OBP != 0]
                if j == 0:
                    continue
                elif data_.iloc[j,:]['OBP'] == 0:
                    continue
                elif len(pre_OBP) == 0:
                    continue
                up_down = data_.iloc[j,:]['OBP'] / pre_OBP[-1]
                if up_down > 1:
                    OBP_up += 1
                elif up_down < 1:
                    OBP_down += 1
                
            data.loc[(data.batter_name==name) & (data.year==year),'OBP_up'] = OBP_up
            data.loc[(data.batter_name==name) & (data.year==year),'OBP_down'] = OBP_down
    
    return data

In [257]:
%%time
data_temp = OBP_up_down(data_temp, names)

Wall time: 18.7 s


## 역대 SLG 상승 수 하락 수

In [258]:
def SLG_up_down(data, names):
    for name in names:
        data_name = data[data.batter_name==name]
        for i in range(data_name.shape[0]):
            SLG_up, SLG_down = 0, 0
            year = data_name.year.iloc[i]
            data_ = data_name.iloc[:i+1,:]
            if data_.shape[0] == 1:
                data.loc[(data.batter_name==name) & (data.year==year),'SLG_up'] = SLG_up
                data.loc[(data.batter_name==name) & (data.year==year),'SLG_down'] = SLG_down
                continue
            for j in range(data_.shape[0]):
                pre_SLG = [SLG for SLG in data_.iloc[:j,:]['SLG'] if SLG != 0]
                if j == 0:
                    continue
                elif data_.iloc[j,:]['SLG'] == 0:
                    continue
                elif len(pre_SLG) == 0:
                    continue
                up_down = data_.iloc[j,:]['SLG'] / pre_SLG[-1]
                if up_down > 1:
                    SLG_up += 1
                elif up_down < 1:
                    SLG_down += 1
                
            data.loc[(data.batter_name==name) & (data.year==year),'SLG_up'] = SLG_up
            data.loc[(data.batter_name==name) & (data.year==year),'SLG_down'] = SLG_down
    
    return data

In [259]:
%%time
data_temp = SLG_up_down(data_temp, names)

Wall time: 18.9 s


# test set

In [39]:
test_set = pd.read_csv('../dataset/submission.csv')

In [40]:
test_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220 entries, 0 to 219
Data columns (total 2 columns):
batter_id      220 non-null int64
batter_name    220 non-null object
dtypes: int64(1), object(1)
memory usage: 3.5+ KB


In [47]:
data1_lst = pd.merge(data1, data1.groupby('batter_name').max()['year'].reset_index(), on=['batter_name','year'], how='right')
data1_lst.shape

(345, 28)

In [51]:
test_set_lst = pd.merge(test_set, data1_lst[['batter_name','team','height/weight','year_born','position','starting_salary']], on='batter_name', how='left') 
test_set_lst.shape

(220, 7)

In [53]:
test_set_lst.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 220 entries, 0 to 219
Data columns (total 7 columns):
batter_id          220 non-null int64
batter_name        220 non-null object
team               220 non-null object
height/weight      220 non-null object
year_born          220 non-null object
position           220 non-null object
starting_salary    177 non-null object
dtypes: int64(1), object(6)
memory usage: 13.8+ KB
